In [13]:
# import needed package
%matplotlib inline
from IPython import display
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import time

import sys
import d2lzh_pytorch as d2l

print(torch.__version__)
print(torchvision.__version__)

1.3.1
0.4.2


In [2]:
#准备数据集
mnist_train = torchvision.datasets.FashionMNIST(root='/mnt1/from_old_machine_P40_M1/zzzzzth/DL/data', train=True, download=True, transform=transforms.ToTensor())
mnist_test = torchvision.datasets.FashionMNIST(root='/mnt1/from_old_machine_P40_M1/zzzzzth/DL/data/', train=False, download=True, transform=transforms.ToTensor())

In [3]:
# show result 
print(type(mnist_train))
print(len(mnist_train), len(mnist_test))

<class 'torchvision.datasets.mnist.FashionMNIST'>
60000 10000


In [4]:
# 可以通过下标来访问任意一个样本
feature, label = mnist_train[0]
print(feature.shape, label)  # Channel x Height x Width

torch.Size([1, 28, 28]) 9


In [5]:
# 显示
mnist_PIL = torchvision.datasets.FashionMNIST(root='/mnt1/from_old_machine_P40_M1/zzzzzth/DL/data', train=True, download=True)
PIL_feature, label = mnist_PIL[1]
print(label)
PIL_feature

0


In [6]:
# 该函数已保存在d2lzh包中
def get_fashion_mnist_labels(labels):
    text_labels = ['t-shirt', 'trouser', 'pullover', 'dress', 'coat',
                   'sandal', 'shirt', 'sneaker', 'bag', 'ankle boot']
    return [text_labels[int(i)] for i in labels]

In [7]:
def show_fashion_mnist(images, labels):
    d2l.use_svg_display()
    # _ 表示我们忽略（不使用）的变量
    _, figs = plt.subplots(1, len(images), figsize=(14, 14))
    for f, img, lbl in zip(figs, images, labels):
        f.imshow(img.view((28, 28)).numpy())
        f.set_title(lbl)
        f.axes.get_xaxis().set_visible(False)
        f.axes.get_yaxis().set_visible(False)
    plt.show()

In [9]:
#有显示bug
# X, y = [], []
# for i in range(10):
#     X.append(mnist_train[i][0]) # 将第i个feature加到X中
#     y.append(mnist_train[i][1]) # 将第i个label加到y中
# show_fashion_mnist(X, get_fashion_mnist_labels(y))

In [10]:
# 读取数据
batch_size = 256
num_workers = 4
train_iter = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=num_workers)
test_iter = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size, shuffle=False, num_workers=num_workers)

In [11]:
#计时
start = time.time()
for X, y in train_iter:
    continue
print('%.2f sec' % (time.time() - start))

1.76 sec


#### 基于softmax训练分类模型

In [15]:
num_inputs = 784
num_outputs = 10

w = torch.tensor(np.random.normal(0, 0.01, (num_inputs, num_outputs)), dtype=torch.float, requires_grad= True)
b = torch.zeros(num_outputs, dtype=torch.float, requires_grad= True)

In [17]:
#tensor求和机制
X = torch.tensor([[1, 2, 3], [4, 5, 6]])
print(X.sum(dim=0, keepdim=True))  # dim为0，按照相同的列求和，并在结果中保留列特征
print(X.sum(dim=1, keepdim=True))  # dim为1，按照相同的行求和，并在结果中保留行特征
print(X.sum(dim=0, keepdim=False)) # dim为0，按照相同的列求和，不在结果中保留列特征
print(X.sum(dim=1, keepdim=False)) # dim为1，按照相同的行求和，不在结果中保留行特征

tensor([[5, 7, 9]])
tensor([[ 6],
        [15]])
tensor([5, 7, 9])
tensor([ 6, 15])


In [48]:
def softmax(x):
    x_exp = x.exp()
    partition = x_exp.sum(dim=1, keepdim=True)
#     print("x size is ", x_exp.size())
#     print("partition size is ", partition, partition.size())
    return x_exp / partition  # 这里应用了广播机制

In [50]:
x = torch.rand((2, 5))
x_prob = softmax(x)
print(x_prob, '\n', x_prob.sum(dim=1))

tensor([[0.1975, 0.1894, 0.2217, 0.2782, 0.1132],
        [0.1335, 0.1965, 0.2372, 0.2517, 0.1811]]) 
 tensor([1.0000, 1.0000])


In [51]:
def net(x):
    return softmax(torch.mm(x.view((-1, num_inputs)), w) + b)

In [52]:
y_hat = torch.tensor([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]])
y = torch.LongTensor([0, 2])
y_hat.gather(1, y.view(-1, 1))

tensor([[0.1000],
        [0.5000]])

In [53]:
y_hat = torch.tensor([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]])
y1 = torch.LongTensor([0, 2])
print(y_hat.gather(1, y1.view(-1,1))) #按行
y2 = torch.LongTensor([0, 1, 1])
print(y_hat.gather(0, y2.view(1,-1))) #按列

tensor([[0.1000],
        [0.5000]])
tensor([[0.1000, 0.2000, 0.5000]])


In [54]:
#交叉熵损失函数
def cross_entropy(y_hat, y):
    return - torch.log(y_hat.gather(1, y.view(-1, 1)))

In [55]:
#准确率
def accuracy(y_hat, y):
    return (y_hat.argmax(dim=1) == y).float().mean().item()

In [56]:
# 已写在d2lzh_pytorch包中方便以后使用
def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0.0, 0
    for x, y in data_iter:
        acc_sum += (net(x).argmax(dim=1) == y).float().sum().item()
        n += y.shape[0]
    return acc_sum / n

In [57]:
print(evaluate_accuracy(test_iter, net))

0.1115


In [58]:
num_epochs, lr = 5, 0.1

# 函数已保存在d2lzh_pytorch包中方便以后使用
def train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size,
              params=None, lr=None, optimizer=None):
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n = 0.0, 0.0, 0
        for X, y in train_iter:
            y_hat = net(X)
            l = loss(y_hat, y).sum()
            
            # 梯度清零
            if optimizer is not None:
                optimizer.zero_grad()
            elif params is not None and params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            
            l.backward()
            if optimizer is None:
                d2l.sgd(params, lr, batch_size)
            else:
                optimizer.step() 
            
            
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().item()
            n += y.shape[0]
        test_acc = evaluate_accuracy(test_iter, net)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc))

train_ch3(net, train_iter, test_iter, cross_entropy, num_epochs, batch_size, [w, b], lr)

epoch 1, loss 0.7864, train acc 0.749, test acc 0.792
epoch 2, loss 0.5697, train acc 0.814, test acc 0.811
epoch 3, loss 0.5260, train acc 0.825, test acc 0.817
epoch 4, loss 0.5010, train acc 0.832, test acc 0.823
epoch 5, loss 0.4860, train acc 0.836, test acc 0.825
